# How to benchmark using trainer

It seems I can't get the Trainer class to work. Let's try on a dummy model and slowly refine below. 

Okay, it seems that I have figured this out. I was being dumb and using a bad model. 

In [1]:
import torch
import torch.nn as nn
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from transformers import BertTokenizer

In [2]:
from typing import List, Optional, Tuple, Union
from torch.utils.data import DataLoader

In [3]:
from transformers import BertConfig

In [4]:
# Load dataset
dataset = load_dataset('glue', 'sst2')

# I believe this is the tokenizer I used... 
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
from transformers import BertPreTrainedModel

In [6]:
# Define a two attention neural network for binary classification
class SimpleFeedForwardNN(BertPreTrainedModel):
    def __init__(self, hidden_size, num_classes):
        config = BertConfig()
        super(SimpleFeedForwardNN, self).__init__(config)
        
        # super(SimpleFeedForwardNN, self).__init__()


        # Use the same embeddings because i'm fucking lazy 
        self.embedding0 = nn.Embedding(30522, hidden_size, padding_idx=0)
        self.embedding1 = nn.Embedding(2, hidden_size, padding_idx=0)
        self.embedding2 = nn.Embedding(2, hidden_size, padding_idx=0)

        self.fc1 = nn.Linear(hidden_size, 4 * hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4 * hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, 4 * hidden_size)
        self.fc6 = nn.Linear(4 * hidden_size, hidden_size)

        self.fc3 = nn.Linear(hidden_size, num_classes)

        # Dummy attention 
        self.attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=4, batch_first=True)
        self.attention1 = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=4, batch_first=True)

        # Create loss
        self.loss_func = nn.CrossEntropyLoss()

        # self.post_init()

    
    def forward(self, 
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        ):

        
        # Dummy embedding 
        out = self.embedding0(input_ids) #+ self.embedding1(attention_mask) + self.embedding2(token_type_ids)

        # Simple attention 
        attention_output, _ = self.attention(out, out, out)
        out = out + attention_output
        ff = self.fc1(out)
        ff = self.relu(ff)
        ff = self.fc2(ff)
        out = out + ff

        # Second attention? 
        attention_output, _ = self.attention1(out, out, out)
        out = out + attention_output
        ff = self.fc5(out)
        ff = self.relu(ff)
        ff = self.fc6(ff)
        out = out + ff

        out = out[:, 0] # Sample "pooler"
        # print(f'\t Pooler: {out[:, 0:10]=}')

        # "Classifier"
        out = self.fc3(out)
            
        loss = self.loss_func(out.view(-1, 2), labels.view(-1))
        
        return (self.loss_func(out, labels), out)


In [7]:
# Define the model
hidden_size = 256
num_classes = 2
model = SimpleFeedForwardNN(hidden_size, num_classes)

In [8]:
from datasets import load_dataset, load_metric
import numpy as np
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    learning_rate=2e-5,
    # warmup_steps=10,
    # weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Load the accuracy metric
metric = load_metric("accuracy")

# Define the compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()



/tmp/ipykernel_713163/2326431495.py:18: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.649800,0.624788,0.660550
2,0.585700,0.590841,0.682339
3,0.555300,0.573673,0.701835
4,0.538100,0.568584,0.705275
5,0.515700,0.566843,0.713303


TrainOutput(global_step=1320, training_loss=0.5809132323120579, metrics={'train_runtime': 407.1806, 'train_samples_per_second': 827.016, 'train_steps_per_second': 3.242, 'total_flos': 1632395967191040.0, 'train_loss': 0.5809132323120579, 'epoch': 5.0})

# Observations

1. First epochs get's bad results; make sense
2. But it slowly gets better in the third and reaches... at least decent stuff right? Need to be patient lmao. But training loss should slowly decrease.
3. So I should be seeing something like this when doing it for my loaded model.. 
4. Is it the "self.post_init()" that's very important? test it out!
5. Do I need to subclass? Don't need to. 

With self.post_init()
```
Epoch	Training Loss	Validation Loss	Accuracy
1	0.690600	0.698000	0.509174
2	0.685200	0.696884	0.509174
3	0.586300	0.580936	0.728211
4	0.439500	0.515325	0.755734
5	0.430200	0.510323	0.763761
```

With just nn.Module
```
Epoch	Training Loss	Validation Loss	Accuracy
1	0.662100	0.644426	0.626147
2	0.580900	0.588855	0.699541
3	0.543800	0.551802	0.723624
4	0.514000	0.539427	0.732798
5	0.498300	0.536726	0.739679
```

Subclasses, no self.post_init()
```
Epoch	Training Loss	Validation Loss	Accuracy
1	0.656900	0.639153	0.638761
2	0.583000	0.596516	0.675459
3	0.577400	0.579494	0.686927
4	0.539300	0.568600	0.699541
5	0.531300	0.565805	0.696101
```

Okay, so I don't need to do it. So what was wrong with before? 

In [9]:
# Prediction
trainer.predict(tokenized_datasets['test'])



DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})